In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# ---- Generating synthetic data ----
num_points = 100
weight = 0.3
bias = 0.9
X = np.linspace(0, 10, num_points)
y = weight * X + bias

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32).view(-1, 1)  # Reshape for PyTorch
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# ---- Define the Linear Regression Model ----
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.weight = nn.Parameter(torch.tensor([0.3], requires_grad=True))
        self.bias = nn.Parameter(torch.tensor([0.9], requires_grad=True))

    def forward(self, x):
        return self.weight * x + self.bias

# Create model instance
model = LinearRegressionModel()

# Define loss function and optimizer
criterion = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# ---- Training Loop ----
num_epochs = 300
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # Backward pass
    loss.backward()
    optimizer.step()

    # Evaluate every 20 epochs
    if (epoch + 1) % 20 == 0:
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test)
            test_loss = criterion(test_outputs, y_test)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}')

print("Training completed!")

# ---- Make Predictions ----
model.eval()
with torch.no_grad():
    predictions = model(X_test)

# ---- Plot Data ----
plt.figure(figsize=(10, 6))
plt.scatter(X_train.numpy(), y_train.numpy(), color='blue', label='Training Data')
plt.scatter(X_test.numpy(), y_test.numpy(), color='red', label='Testing Data')
plt.scatter(X_test.numpy(), predictions.numpy(), color='green', marker='x', label='Model Predictions')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Training, Testing Data, and Predictions')
plt.legend()
plt.show()

# ---- Save & Load the Model ----
model_path = "linear_regression_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Model's state_dict saved to {model_path}")

# Load the saved model
loaded_model = LinearRegressionModel()
loaded_model.load_state_dict(torch.load(model_path))
loaded_model.eval()

# Verify predictions
with torch.no_grad():
    loaded_predictions = loaded_model(X_test)

print(f"Predictions match: {torch.allclose(predictions, loaded_predictions)}")
